In [1]:
import os
import logging
import globals as g
from dotenv import load_dotenv
from unsloth import to_sharegpt
from datasets import load_dataset
from unsloth import FastLanguageModel
from unsloth import standardize_sharegpt
from ec2_metrics import EC2MetricsCallback

# Create a logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Remove existing handlers
logger.handlers.clear()

# Add a simple handler
handler = logging.StreamHandler()
formatter = logging.Formatter('[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/ubuntu/spring-2025-lab07-Jonah-Lichtenthal/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


2025-03-16 23:42:27,712	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [5]:
# Load environment variables from .env file
import getpass
load_dotenv()
if not os.getenv("HF_TOKEN"):
    os.environ["HF_TOKEN"] = getpass.getpass("Enter your HuggingFace token: ")
hf_token = os.getenv("HF_TOKEN")

if not os.getenv("HF_MODEL_ID"):
    hf_model_id  = input("Enter the model id to use for fine-tuning (e.g. meta-llama/Llama-3.1-8B-Instruct): ")
else:
    hf_model_id = os.getenv("HF_MODEL_ID")
logger.info(f"hf_model_id={hf_model_id}")

[2025-03-16 23:44:07,757] p52095 {2602757443.py:12} INFO - hf_model_id=meta-llama/Llama-3.1-8B-Instruct


In [3]:
from datasets import load_dataset
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

DATASET_OF_INTEREST: str = 'mteb/banking77'

ALPACA_PROMPT: str = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [8]:
try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = hf_model_id,
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
        token = hf_token # use one if using gated models like meta-llama/Llama-2-7b-hf
    )
except Exception as e:
    logger.error(f"Error occurred while loading the model: {e}")
    raise

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.045 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [10]:
dataset = load_dataset(DATASET_OF_INTEREST, split="train")
logger.info(f"Columns in the dataset: {dataset.column_names}")

[2025-03-16 23:46:53,499] p52095 {2022229014.py:2} INFO - Columns in the dataset: ['text', 'label', 'label_text']


In [12]:
dataset = load_dataset(DATASET_OF_INTEREST, split="train")
logger.info(f"Columns in the dataset: {dataset.column_names}")
dataset = to_sharegpt(
    dataset,
    merged_prompt="Below is a user query:\n{text}\n\nWhat is the correct category?",
    output_column_name="label_text", # Expected output is the label text
    conversation_extension=3,
)

[2025-03-16 23:48:29,250] p52095 {3255405979.py:2} INFO - Columns in the dataset: ['text', 'label', 'label_text']


In [13]:
dataset = standardize_sharegpt(dataset)

In [14]:
from pprint import pprint
pprint(dataset[:3])

{'conversations': [[{'content': 'Below is a user query:\n'
                                "('I am still waiting on my card?',)\n"
                                '\n'
                                'What is the correct intent?',
                     'role': 'user'},
                    {'content': 'card_arrival', 'role': 'assistant'},
                    {'content': 'Below is a user query:\n'
                                "('How can I convert currencies?',)\n"
                                '\n'
                                'What is the correct intent?',
                     'role': 'user'},
                    {'content': 'exchange_via_app', 'role': 'assistant'},
                    {'content': 'Below is a user query:\n'
                                "('Will be Apple Watch be able to let me top "
                                "up?',)\n"
                                '\n'
                                'What is the correct intent?',
                     'role': 'user'},


In [15]:
chat_template = """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.

### Instruction:
{INPUT}

### Response:
{OUTPUT}"""

from unsloth import apply_chat_template

dataset = apply_chat_template(
    dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

Unsloth: We automatically added an EOS token to stop endless generations.
Map: 100%|██████████| 10003/10003 [00:00<00:00, 17151.92 examples/s]


In [16]:
%%time
# train the model
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        # num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
    callbacks=[EC2MetricsCallback],
)

Converting train dataset to ChatML (num_proc=2): 100%|██████████| 10003/10003 [00:00<00:00, 18835.80 examples/s]
Applying chat template to train dataset (num_proc=2): 100%|██████████| 10003/10003 [00:01<00:00, 6753.93 examples/s]
Tokenizing train dataset (num_proc=2): 100%|██████████| 10003/10003 [00:00<00:00, 10313.22 examples/s]

CPU times: user 876 ms, sys: 377 ms, total: 1.25 s
Wall time: 6.42 s


In [ ]:
%%time
# this will initiate the training process and also log the EC2 utilization metrics, such as the GPU
# utilization, CPU utilization, etc.
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 10,003 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
[2025-03-16 23:49:46,788] p52095 {ec2_metrics.py:184} INFO - Training started. Initiating EC2 metrics collection.
[2025-03-16 23:49:46,790] p52095 {ec2_metrics.py:170} INFO - Writing header: ['timestamp', 'cpu_percent_mean', 'memory_percent_mean', 'memory_used_mean', 'gpu_utilization_mean', 'gpu_memory_used_mean', 'gpu_memory_free_mean', 'gpu_memory_total_mean']
[2025-03-16 23:49:46,791] p52095 {ec2_metrics.py:41} INFO - Starting collection
[2025-03-16 23:49:47,135] p52095 {ec2_metrics.py:143} INFO - Starting daemon collector to run in background


Step,Training Loss
1,2.602700
2,2.675300
3,2.756200
4,2.622100
5,2.462100
6,2.221900
7,1.982400
8,1.789600
9,1.554300
10,1.535700


[2025-03-16 23:52:33,048] p52095 {ec2_metrics.py:191} INFO - Training ended. Stopping EC2 metrics collection.
[2025-03-16 23:52:33,049] p52095 {ec2_metrics.py:33} INFO - Stopped collection


CPU times: user 1min 43s, sys: 1min 5s, total: 2min 48s
Wall time: 2min 47s


[2025-03-16 23:52:37,186] p52095 {ec2_metrics.py:33} INFO - Stopped collection


In [18]:
# Format the training stats in a readable way
output_text = f"""Training Statistics:
Global Steps: {trainer_stats.global_step}
Training Loss: {trainer_stats.training_loss:.4f}

Metrics:
- Train Runtime: {trainer_stats.metrics['train_runtime']:.3f} seconds
- Training Samples/Second: {trainer_stats.metrics['train_samples_per_second']:.3f}
- Training Steps/Second: {trainer_stats.metrics['train_steps_per_second']:.3f}
- Total FLOPS: {trainer_stats.metrics['total_flos']:.2e}
- Final Train Loss: {trainer_stats.metrics['train_loss']:.4f}
"""

# Save to a text file
with open(os.path.join(g.RESULTS_DIR, g.TRAINING_STATS), 'w') as f:
    f.write(output_text)

In [19]:
# save the model
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [23]:
questions = ["I see a charge on my credit card statement but I paid on time, why?",
"Do you have a branch in Timbuktu?",
"I lost my card and my replacement card has not arrived."]

In [25]:
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": questions[0]},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    input_ids,
    max_new_tokens=128,
    pad_token_id=tokenizer.eos_token_id
)

response = tokenizer.decode(outputs[0][input_ids.shape[-1]:])
q1 = f"""input: {questions[0]}\ncategory: {response}"""
q1
        

"input: I see a charge on my credit card statement but I paid on time, why?\ncategory: If you think there's a charge you didn't make, you can dispute it by going to the 'Transaction Dispute' section in the app.<|eot_id|>"

In [27]:
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": questions[1]},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    input_ids,
    max_new_tokens=128,
    pad_token_id=tokenizer.eos_token_id
)

response = tokenizer.decode(outputs[0][input_ids.shape[-1]:])
q2 = f"""input: {questions[1]}\ncategory: {response}"""
q2

'input: Do you have a branch in Timbuktu?\ncategory: Yes, there is a branch in Timbuktu.<|eot_id|>'

In [28]:
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": questions[2]},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(
    input_ids,
    max_new_tokens=128,
    pad_token_id=tokenizer.eos_token_id
)

response = tokenizer.decode(outputs[0][input_ids.shape[-1]:])
q3 = f"""input: {questions[2]}\ncategory: {response}"""
q3

'input: I lost my card and my replacement card has not arrived.\ncategory: What is the estimated delivery time for my replacement card?<|eot_id|>'

In [30]:
with open("problem1_task1.txt", 'w') as outfile:
    outfile.write(f"{q1}\n{q2}\n{q3}")